In [15]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, Dataset
import pandas as pd
from sklearn.model_selection import train_test_split

RANDOM_SEED = 69

In [21]:
# Check distribution of dataset

data = pd.read_csv('Datasets/Cleaned/finetuning_dataset.csv')
twitter_data = data[data['source'] == 'Twitter'].sample(frac=0.3, random_state=RANDOM_SEED)
other_data = data[data['source'] != 'Twitter']
sampled_data = pd.concat([twitter_data, other_data])

print(sampled_data['source'].value_counts().sum())

387528


In [30]:
# Perform train-test split
train_df, eval_df = train_test_split(
    sampled_data, 
    test_size=0.1, 
    stratify=sampled_data['source'],  # Preserve source distribution
    random_state=RANDOM_SEED
)

train_dataset = Dataset.from_pandas(train_df)
eval_dataset = Dataset.from_pandas(eval_df)

# Inspect the splits
print(f"Training size: {len(train_dataset)}, Evaluation size: {len(eval_dataset)}")

Training size: 348775, Evaluation size: 38753


In [31]:
# Load Tokenizer and Model
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
# Tokenize Dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_eval = eval_dataset.map(tokenize_function, batched=True)

tokenized_train = tokenized_train.map(lambda examples: {'labels': torch.tensor(examples['polarity'], dtype=torch.long)})
tokenized_eval = tokenized_eval.map(lambda examples: {'labels': torch.tensor(examples['polarity'], dtype=torch.long)})

Map: 100%|██████████| 38753/38753 [00:01<00:00, 20609.38 examples/s]


In [33]:
tokenized_train

Dataset({
    features: ['text', 'polarity', 'source', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 348775
})

In [34]:
tokenized_eval

Dataset({
    features: ['text', 'polarity', 'source', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 38753
})

In [35]:
# Inspect the First Row of Tokenized Train Dataset
row = tokenized_train[0]

# Print `input_ids`
print("Input IDs:", row['input_ids'])

# Decode Back to Text (Optional)
decoded_text = tokenizer.decode(row['input_ids'], skip_special_tokens=True)
print("Decoded Text:", decoded_text)

Input IDs: [101, 1996, 5790, 1045, 2435, 2323, 2941, 2022, 1037, 5717, 2349, 2000, 3532, 8013, 2326, 1012, 2044, 8110, 2041, 1996, 2433, 2006, 1996, 4037, 2445, 2006, 2026, 3025, 3319, 2035, 1045, 2363, 2001, 1037, 7514, 2011, 1000, 6207, 1000, 1999, 8013, 2326, 4129, 2033, 2000, 2655, 1012, 1045, 1005, 2310, 2525, 5287, 2055, 1996, 3573, 1012, 2023, 3006, 2006, 10688, 27136, 2015, 1010, 6719, 1998, 29407, 2135, 1012, 8013, 2326, 2005, 2023, 2194, 2003, 2512, 4839, 4630, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Decoded Text: the rating i gave should actually be a zero due to poor customer service. after filling out the form on the website given on my previous review all i received was a reply by " apple " in customer service telling me to call. i've already spoken about the store. this market on speedway stinks, literally and figuratively. customer service for this compa

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
batch_size = 16
epochs = 3
total_steps = (len(tokenized_train) // batch_size) * epochs

In [ ]:
training_args = TrainingArguments(
    output_dir="./basemodel_results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=epochs,
    weight_decay=0.01,
    warmup_steps=int(0.1 * total_steps),
    save_strategy="epoch",
    save_total_limit=10,
    logging_dir="./logs",
    report_to="none",
    logging_steps=500,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

trainer.evaluate()

model.save_pretrained('./basemodel_sentiment_model')
tokenizer.save_pretrained('./basemodel_sentiment_model')